# Assignment 2 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [1]:
# Part 1

# Fill in the answer to part 1 below as a decimal number
problem1_p1 = 0.5*0.2+0.2*0.3+0.3*0.4
problem1_p1

0.28

In [2]:
# Part 2

# Fill in the answer to part 2 below as a decimal number
problem1_p2 = 0.5*0.2+0.3*0.4
problem1_p2

0.22

In [3]:
# Part 3

# Fill in the answer to part 3 below as a boolean
problem1_irreducible = True

In [4]:
# Part 4

# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
import numpy as np
import numpy.linalg as linalg

P = np.array([
    [0.3, 0.4, 0.3],
    [0.2, 0.5, 0.3],
    [0.4, 0.3, 0.3]
])
w, vl = linalg.eig(P.T)
problem1_stationary = list(vl[:,0]/np.sum(vl[:, 0]))

In [5]:
# Part 5

# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
STATE = 0
NUMBER_OF_STATES = 3
steps = 0

MFPT = np.zeros((3,3))
for i in range(3):
        for j in range(NUMBER_OF_STATES):
            if j != i:
                MFPT[i, j] = 1
                for _ in range(30):
                    MFPT[i, j] = 1 + sum(P[i, k] * MFPT[k, j] for k in range(3) if k != j)
problem1_ET = 3.846

---
## Assignment 2, PROBLEM 2
Maximum Points = 8


A healthcare organization is interested in understanding the relationship between the number of visits to the doctors office and certain patient characteristics. 
They have collected data on the number of visits (for a year) for a sample of patients and have included the following variables

* ofp : number of physician office visits
* ofnp : number of nonphysician office visits
* opp : number of physician outpatient visits
* opnp : number of nonphysician outpatient visits
* emr : number of emergency room visits
* hosp : number of hospitalizations
* exclhlth : the person is of excellent health (self-perceived)
* poorhealth : the person is of poor health (self-perceived)
* numchron : number of chronic conditions
* adldiff : the person has a condition that limits activities of daily living ?
* noreast : the person is from the north east region
* midwest : the person is from the midwest region
* west : the person is from the west region
* age : age in years (divided by 10)
* male : is the person male ?
* married : is the person married ?
* school : number of years of education
* faminc : family income in 10000$
* employed : is the person employed ?
* privins : is the person covered by private health insurance?
* medicaid : is the person covered by medicaid ?

Decide which patient features are resonable to use to predict the target "number of physician office visits". Hint: should we really use the "ofnp" etc variables?

Since the target variable is counts, a reasonable loss function is to consider the target variable as Poisson distributed where the parameter follows $\lambda = \exp(\alpha \cdot x + \beta)$ where $\alpha$ is a vector (slope) and $\beta$ is a number (intercept). That is, the parameter is the exponential of a linear function. The reason we chose this as our parameter, is that it is always positive which is when the Poisson distribution is defined. To be specific we make the following assumption about our conditional density of $Y \mid X$,
$$
    f_{Y \mid X} (y,x) = \frac{\lambda^{y} e^{-\lambda}}{y !}, \quad \lambda(x) = \exp(\alpha \cdot x + \beta).
$$

Recall from the lecture notes, (4.2) that in this case we should consider the log-loss (entropy) and that according to (4.2.1 Maximum Likelihood and regression) we can consider the conditional log-likelihood. Follow the steps of Example 1 and Example 2 in section (4.2) to derive the loss that needs to be minimized.

Hint: when taking the log of the conditional density you will find that the term that contains the $y!$ does not depend on $\lambda$ and as such does not depend on $\alpha,\beta$, it can thus be discarded. This will be essential due to numerical issues with factorials.

Instructions:

1. Load the file `data/visits_clean.csv`, follow the instructions in the code cell of how this should happen [1.5p]
2. Create the `problem2_X` and the `problem2_y` as numpy arrays with `problem2_X` being the features and `problem2_y` being the target. Do the standard train-test split with 80% training data and 20% testing data. Store these in the variables defined in the cells. [1.5p]
3. Implement $loss$ inside the class `PoissonRegression` by writing down the loss to be minimized, I have provided a formula for the $\lambda$ that you can use. [1.5p]
4. Now use the `PoissonRegression` class to train a Poisson regression model on the training data. [1.5p]
5. Compute the mean absolute error of your prediction on the test set and use Hoeffdings inequality to produce a 95\% confidence interval for the mean absolute error. We can make the assumption that the error is bounded by 70 for simplicity. [2p]


In [6]:
# Part 1

# As in assignment 1 we will load the header into header and data into data
# this time you will have to parse the data such that each data entry is a float
# and that the problem2_data is a numpy array of shape (n_samples,n_columns)
# where n_columns is the number of columns and should have the same length as
# the list of strings header. n_samples is how many rows of data we had.
# If you cannot find the file, check the starting package as it should be updated
# if not, go to the github repo and pull it

# The autograder does not accept pandas as a solution to this problem.
import numpy as np
import csv
path = "../data/visits_clean.csv"

with open(path, mode="r") as file:
    csv_reader = csv.DictReader(file, delimiter=" ")
    problem2_header = csv_reader.fieldnames
problem2_data = np.loadtxt(path, delimiter=" ", skiprows=1) #A numpy array of shape n_samples n_columns
print(len(problem2_header))
print(problem2_data.shape)

21
(4406, 21)


In [7]:
# Part 2
corr_ind = []
corrs = np.corrcoef(problem2_data, rowvar=False)[:,0]
threshold = corrs[1]
for i, val in enumerate(corrs[2:]):
    if val > threshold:
        corr_ind.append(i+2)
# Fill in your X and y below
problem2_X = problem2_data[:, corr_ind]
problem2_y = problem2_data[:, 0]

# Split the data into train and randomly using for instance
# np.random.shuffle indices and indexing the first 80% as the train data
# keep the train size as 0.8 rounded up to the nearest integer sample
# np.random.shuffle(problem2_data)
n_split = int(np.ceil(0.8*problem2_data.shape[0]))
problem2_X_train, problem2_X_test, problem2_y_train, problem2_y_test = problem2_data[:n_split, corr_ind], problem2_data[n_split:, corr_ind], problem2_data[:n_split,0], problem2_data[n_split:,0]
problem2_X_train.shape

(3525, 2)

In [8]:
# Part 3

# Fill in the function loss below

class PoissonRegression(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        # The parameter lambda for the given X and the proposed values 
        # of the coefficients, here coeff[:-1] represent alpha 
        # and coeff[-1] represent beta
        lam = np.exp(X.dot(coeffs[:-1]) + coeffs[-1])
        # use the Y variable that is available here to define 
        # the loss function, return the value of the loss for 
        # this Y and for this parameter lam defined above
        #print(Y)
        loss = - np.sum((np.multiply(Y, np.log(lam)) - lam)/np.size(Y))
        # print(loss)
        return loss

    def fit(self,X,Y):
        import numpy as np
        from scipy import optimize

        #Use the loss above together with an optimization method from scipy
        #to find the coefficients of the model
        #this is prepared for you below
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1) # initial guess as 0
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x

    
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            return np.exp(np.dot(X,self.coeffs[:-1])+self.coeffs[-1])

In [9]:
# Part 4

# Initialize your PoissonRegression model
problem2_model = PoissonRegression()

# Fit your initialized model on the training data
problem2_model.fit(problem2_X_train, problem2_y_train)

# This is to make sure that everything went well, 
# check that success is True
print(problem2_model.result)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -4.712031000962939
       x: [ 1.892e-01  1.746e-01  1.379e+00]
     nit: 28
     jac: [-5.782e-06  6.497e-06 -1.788e-07]
    nfev: 232
    njev: 58


In [10]:
# Part 5

# Put the computed mean absolute error in the variable below
problem2_metric = np.mean(np.abs(problem2_y_test - problem2_model.predict(problem2_X_test)))
print(problem2_metric)
# Put a confidence interval in the variable below by using Hoeffdings inequality using the bounds
# a = 0, b=70 (roughly 5 days between visits as minimum)
# the variable should contain a tuple representing the confidence interval of the form (l_edge,r_edge)
n = problem2_X_test.shape[0]
a = 0
b = 70

delta = 0.05

epsilon = np.sqrt((b - a)**2 * np.log(2 / delta) / (2 * n))

problem2_interval = (problem2_metric - epsilon, problem2_metric + epsilon)
epsilon

4.116213259221614


3.202892184529445

---
## Assignment 2, PROBLEM 3
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [11]:

def problem3_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    m = 2**32
    a = 1664525
    b = 1013904223
    x = seed # the seed
    lst = []
    for i in range(size):
        x = (a * x + b) % m
        lst.append(x)
    return lst
# len(problem3_LCG(10))

In [12]:

def problem3_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    nums = generator(size, seed)
    lst = [x/(period-1) for x in nums]
    
    return lst

In [13]:

def problem3_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution.
    Runs n_iterations
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    n_iterations : an integer denoting how many attempts should be made in the accept-reject sampler
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    samples = []
    x = uniformGenerator(size=n_iterations, seed=seed)
    y = uniformGenerator(size=n_iterations, seed=seed)
    for x,y in zip(x,y):
        if (y*(np.pi/2)) <= ((np.pi/2)*np.abs(np.sin(x*2*np.pi))):
            samples.append(x)
    
    return samples

---
#### Local Test for Assignment 2, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [14]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem3_LCG(size=10, seed = 1))

period = 2**32

print("Uniform sampler %s" % problem3_uniform(generator=problem3_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem3_uniform(generator=problem3_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem3_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [1015568748, 1586005467, 2165703038, 3027450565, 217083232, 1587069247, 3327581586, 2388811721, 70837908, 2745540835]
Uniform sampler [0.23645552532664862, 0.3692706738061436, 0.5042420324180839, 0.7048832638433397, 0.050543628644790416, 0.3695183543883074, 0.7747629626595329, 0.5561885706978358, 0.01649323571857373, 0.6392460399398687]
Accept-Reject sampler [0.23645552532664862, 0.3692706738061436, 0.7048832638433397, 0.050543628644790416, 0.3695183543883074, 0.7747629626595329, 0.01649323571857373, 0.6392460399398687, 0.2504511415610209, 0.4223777683038213, 0.8369336593516482, 0.23507591947798523, 0.3268755030182366, 0.6826027218910407]


In [15]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem3_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.13436424411240122, 0.763774618976614, 0.2550690257394217, 0.651592972722763, 0.7887233511355132, 0.0938595867742349, 0.02834747652200631, 0.8357651039198697, 0.762280082457942, 0.0021060533511106927, 0.7215400323407826, 0.22876222127045265, 0.030589983033553536]
